# Spoken Language Processing 2022-23

# Lab3 - Dialogue Systems

_Bruno Martins_


This lab assignment will introduce tools and concepts related to the development of dialogue systems, exemplifying also the use of automatic speech recognition and text-to-speech models.

Students will be tasked with the development of a simple (spoken/conversational) question answering system, reusing different models associated to the HuggingFace Transformers library:

* Speech recognition models (e.g., OpenAI Whisper).
* Large language models for natural language understanding and generation (e.g., GPT-2 or Alpaca models).
* Text-to-speech models (e.g., SpeechT5).

The first parts of this notebook will guide students in the use of the tools, while the last part presents the main problem that is to be tackled. Note that the first parts also features intermediate tasks, which students are required to solve.

To complete the project, student groups must deliver in Fenix an updated version of this notebook, featuring the proposed solutions to each task, together with a small PDF report (2 pages) outlining the methods that were developed (you can use the [following Overleaf template](https://www.overleaf.com/latex/templates/interspeech-2023-paper-kit/kzcdqdmkqvbr) for the report).

Students are encouraged to modify examples, incorporate any other techniques, and in general explore any approach that may permit improving the results. Assessment will be based on task completion, creativity in the proposed solutions, and overall accuracy over a benchmark dataset.

### Group identification

Initialize the variable `group_id` with the number that Fenix assigned to your group and `student1_name`, `student1_id`, `student2_name` and `student2_id` with your names and student numbers.

In [45]:
# # YOUR CODE HERE
# raise NotImplementedError()
group_id = 9
student1_name = "Afonso Araújo"
student1_id = 96138
student2_name = "Santiago Quintas"
student2_id = 93179
print(f"Group number: {group_id}")
print(f"Student 1: {student1_name} ({student1_id})")
print(f"Student 2: {student2_name} ({student2_id})")

Group number: 9
Student 1: Afonso Araújo (96138)
Student 2: Santiago Quintas (93179)


In [46]:
assert isinstance(group_id, int) and isinstance(student1_id, int) and isinstance(student2_id, int)
assert isinstance(student1_name, str) and isinstance(student2_name, str) 
assert (group_id > 0) and (group_id < 40)
assert (student1_id > 60000) and (student1_id < 120000) and (student2_id > 60000) and (student2_id < 120000)

# Python packages

NumPy is a Python library that provides functions to process multidimensional array objects. The NumPy documentation is available [here](https://numpy.org/doc/1.24/).

[Librosa](https://librosa.org/) is a Python package for analyzing and processing audio signals. It provides a wide range of tools for tasks such as loading and manipulating audio files, extracting features from audio signals, and visualizing and playing back audio data.

IPython display is a module in the IPython interactive computing environment that provides a set of functions for displaying various types of media in the Jupyter notebook or other IPython-compatible environments. For example, you can use the display() function to display an object in a notebook cell (for example an audio object).

Matplotlib is a popular Python library that allows users to create a wide range of visualizations using a simple and intuitive syntax.

Huggingface transformers provides APIs and tools to easily download and train state-of-the-art pretrained models based on the Transformer architecture. The documentation is available [here](https://huggingface.co/docs/transformers/index) and, for more details, look at the official [HuggingFace course](https://huggingface.co/course/chapter1/1).

The associated HuggingFace libraries named [datasets](https://huggingface.co/docs/datasets/index) and [evaluate](https://huggingface.co/docs/evaluate/index) respectivly suport the direct access to many well-known datasets and common evaluation metrics used in NLP and speech research.

In [47]:
# pip install sentencepiece
# pip install xformers
# pip install transformers
# pip install datasets
# pip install evaluate
# pip install jiwer
# pip install librosa

In [48]:
import transformers
import datasets
import evaluate
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio
from matplotlib import pyplot as plt

# Using OpenAI Whisper

Whisper is an exciting new model for Automatic Speech Recognition (ASR), developed by OpenAI and made available through the HuggingFace Transformers library.

The following example illustrates the use of the Whisper model to transcribe a small audio sample taken from the LibriSpeech dataset (which is available through the HuggingFace datasets library).

More detailed information about Whisper, including information on how to fine-tune the model with task-specific data, is available on a [tutorial in the HuggingFace blog](https://huggingface.co/blog/fine-tune-whisper).

In [49]:
import torch
import librosa
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

processor_whisper = AutoProcessor.from_pretrained("openai/whisper-tiny.en")
model_whisper = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

audio = ds[0]["audio"]["array"]
audio = librosa.resample(audio, orig_sr=16000, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor_whisper(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(audio, rate=16000)) # You are able to hear the audio inputs

generated_ids = model_whisper.generate(inputs=input_features)
transcription = processor_whisper.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

Found cached dataset librispeech_asr_dummy (C:/Users/afons/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


In [50]:
## openai/whisper-base.en

In [51]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor_base = WhisperProcessor.from_pretrained("openai/whisper-base.en")
model_base = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base.en")

# load dummy dataset and read audio files
audio = ds[0]["audio"]["array"]
audio = librosa.resample(audio, orig_sr=16000, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor_base(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(audio, rate=16000)) # You are able to hear the audio inputs

generated_ids = model_base.generate(inputs=input_features)
transcription = processor_base.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


In [52]:
## facebook/s2t-small-librispeech-asr

In [53]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset

model_facebook = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor_facebook = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

audio = ds[0]["audio"]["array"]
audio = librosa.resample(audio, orig_sr=16000, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor_facebook(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(audio, rate=16000)) # You are able to hear the audio inputs

generated_ids = model_facebook.generate(inputs=input_features)
transcription = processor_facebook.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

c:\Users\afons\anaconda3\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


mister quilter is the apostle of the middle classes and we are glad to welcome his gospel


In [54]:
## facebook/wav2vec2-base-960h

In [55]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and tokenizer
processor_wav2vec = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_wav2vec = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

audio = ds[0]["audio"]["array"]
audio = librosa.resample(audio, orig_sr=16000, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor_wav2vec(audio, sampling_rate=16000, return_tensors="pt")
logits = model_wav2vec(inputs.input_values).logits

display(Audio(audio, rate=16000)) # You are able to hear the audio inputs

generated_ids = torch.argmax(logits, dim=-1)
transcription = processor_wav2vec.batch_decode(generated_ids)
transcription = transcription[0].replace("<s>", "").replace("</s>", "")

print(transcription)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL


Automatic Speech Recognition (ASR) models are frequently evaluated through the Word Error Rate (WER).

The WER is derived from the Levenshtein distance, working at the word level and aligning the recognized word sequence with the reference (spoken) word sequence using dynamic string alignment. The metric can then be computed as:

WER = (S + D + I) / N = (S + D + I) / (S + D + C),

where S is the number of substitutions, D is the number of deletions, I is the number of insertions, C is the number of correct words, and N is the number of words in the reference (N=S+D+C). The WER value indicates the average number of errors per reference word. The lower the value, the better the performance of the ASR system, with a WER of 0 being a perfect score.

The example below illustrates the computation of the WER for two paired examples of a generated sentence versus a reference sentence. The score produced as output is the average value accross the two examples.

In [56]:
from evaluate import load

wer = load("wer")
predictions = ["this is the prediction", "there is an other sample"]
references = ["this is the reference", "there is another one"]
wer_score = wer.compute(predictions=predictions, references=references)

print(wer_score)

0.5


## Intermediate tasks:

* Collect two small audio samples with your own voice, together with a transcription of the spoken messages. The following [example shows how to record audio from your microphone within a Python notebook running on Google Colab](https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb#scrollTo=H4rxNhsEpr-c), but you can use any other method to collect the audio samples.
* Use the Whisper speech recognition model to transcribe the two spoken messages that were collected.
* Use the transcriptions to compute the word error rate.
* Experiment with the use of different recognition models (e.g., larger Whisper models), and see if the error rate changes.

In [57]:
audio1,_ = librosa.load("helloHowRu.wav", sr=16000)
audio1 = librosa.resample(audio1, orig_sr=22050, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)
audio2,_ = librosa.load("whereRuFrom.wav", sr=16000)
audio2 = librosa.resample(audio2, orig_sr=22050, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)
audio_names = [audio1, audio2]

# Testing ASR with OpenAI Whisper Base (100M Parameters)

predictions = []
references = ["Hello how are you?", "Where are you from?"]
for audio in audio_names:
    inputs = processor_base(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_base.generate(inputs=input_features)
    transcription = processor_base.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]
    predictions.append(transcription)
    print(f"Transcription is {transcription}")

wer_score = wer.compute(predictions=predictions, references=references)
print(f"For openai/whisper.base the Word Error Rate score is {wer_score}")


# Testing ASR with OpenAI Whisper Tiny (39M Parameters)

predictions = []
references = ["Hello how are you?", "Where are you from?"]

for audio in audio_names:
    inputs = processor_whisper(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]
    predictions.append(transcription)
    print(f"Transcription is {transcription}")

wer_score = wer.compute(predictions=predictions, references=references)
print(f"For openai/whisper.tiny the Word Error Rate score is {wer_score}")

# Testing ASR with Facebook S2T (1000 Hours training data)

predictions = []
for audio in audio_names:
    inputs = processor_facebook(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_facebook.generate(inputs=input_features)
    transcription = processor_facebook.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]
    predictions.append(transcription)
    print(f"Transcription is {transcription}")

wer_score = wer.compute(predictions=predictions, references=references)
print(f"For facebook/s2t-small-librispeech-asr the Word Error Rate score is {wer_score}")

# Testing ASR with Facebook Wav2Vec2 (960 Hours training data)

predictions = []
for audio in audio_names:
    inputs = processor_wav2vec(audio, sampling_rate=16000, return_tensors="pt")
    logits = model_wav2vec(inputs.input_values).logits
    generated_ids = torch.argmax(logits, dim=-1)
    transcription = processor_wav2vec.batch_decode(generated_ids)
    transcription = transcription[0].replace("<s>", "").replace("</s>", "")
    predictions.append(transcription)
    print(f"Transcription is {transcription}")

wer_score = wer.compute(predictions=predictions, references=references)
print(f"For facebook/wav2vec2-base-960h the Word Error Rate score is {wer_score}")





Transcription is  Hello, how are you?
Transcription is  Where are you from?
For openai/whisper.base the Word Error Rate score is 0.125
Transcription is  Hello, how are you?
Transcription is  Where are you from?
For openai/whisper.tiny the Word Error Rate score is 0.125
Transcription is the yellow are hyacinth
Transcription is then where are you from replied the little girl
For facebook/s2t-small-librispeech-asr the Word Error Rate score is 1.25
Transcription is HELLO HOW ARE YOU
Transcription is THEN WHERE ARE YOU FROM NOTECUNELE
For facebook/wav2vec2-base-960h the Word Error Rate score is 1.25


# Using LLMs for conditional language generation

OpenAI GPT-2 is a large transformer-based language model with 1.5 billion parameters, trained on a dataset of 8 million web pages. GPT-2 is trained with a simple objective: predict the next word, given all of the previous words within some text. The diversity of the dataset causes this simple goal to contain naturally occurring demonstrations of many tasks across diverse domains. Thus, GPT-2 can be used to address problems like question answering, modeling the task as language generation conditioned in the question (plus other relevant additional context).

The following example illustrates the use of the GPT-2 through the Huggingface Transformers library. In this case, instead of using the model directly, we are using the model through the pipeline API, which facilitates the adaptation to the case of other LLMs. The pipeline() function can be used to connect a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer.

In [58]:
from transformers import pipeline, set_seed

set_seed(42) # make results deterministic

generator = pipeline(model='gpt2')
generator("Who is the president of the United States? The answer is", max_length=15, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Who is the president of the United States? The answer is: Donald Trump'}]

## Intermediate tasks:

* Adapt the example showing how to use GPT-2 to do question answering over the [SQuAD dataset](https://rajpurkar.github.io/SQuAD-explorer/) (available from HuggingFace datasets).
* Evaluate the results obtained with different models (e.g., [Alpaca-based models](https://huggingface.co/declare-lab/flan-alpaca-base)) and/or different usage strategies (e.g., consider prompting, parameter efficient fine-tuning, etc.).
* Compute the error over the first 1000 examples from the validation split from the SQuAD dataset, using the [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu) for comparing the generated answers against the ground truth.


In [59]:
# Access SQuAD dataset
ds = load_dataset("squad", split="validation")
bleu = evaluate.load("bleu")

sample_size = 2

# Questions/Answers/Contexts from SQuAD dataset

questions_vec = [ds[i]["question"] for i in range(sample_size)]
answers_vec = [ds[i]["answers"]["text"] for i in range(sample_size)]
contexts_vec = [ds[i]["context"] for i in range(sample_size)]

model_names = ['gpt2', 'declare-lab/flan-alpaca-base', 'facebook/opt-350m']
predictions = {model_name: [] for model_name in model_names}

# References will hold 3 references for each question due to nature of SQuAD dataset
# Hence the need for 3 predictions for each model
references = {model_name: [] for model_name in model_names}

beg = "I have  a question for you. The question is: "
middle = "To help you answer the question, I will give you some context. The context is: "
end = "Answer the question with 3 words or less. The answer is "

for i in range(sample_size):
    print("In sample number " + str(i) + " out of " + str(sample_size) + " samples in total")
    for model_name in model_names:
        references[model_name].append(answers_vec[i])
        # Create prompt for each model
        prompt = beg + questions_vec[i] + middle + contexts_vec[i] + end
        # Generate answer
        generator = pipeline(model=model_name)
        generated_answer = generator(prompt, max_length=220, num_return_sequences=1)
        # if model_name is gpt-2 strip away the prompt from the genrated_answer
        if model_name == 'gpt2':
            answer = generated_answer[0]['generated_text'][len(prompt):len(generated_answer[0]['generated_text'])]    
        # if model_name is opt-350m strip away the prompt from the genrated_answer
        if model_name == 'facebook/opt-350m':
            answer = generated_answer[0]['generated_text'][len(prompt):len(generated_answer[0]['generated_text'])]
        if model_name == 'declare-lab/flan-alpaca-base':
            answer = generated_answer[0]['generated_text']
        # answer = answer.replace('\n', ' ').strip()
        # if answer == '':
        #     answer = "?"
        predictions[model_name].append(answer)

# more post-processing
for i in range(sample_size):
    print("Currently in sample ", i)
    for model_name in model_names:
        print("Pre process prediction for", model_name,"is - ", predictions[model_name][i])
        # if predictions[model_name][i] has -, _, or \n remove it then start from the first letter or number
        # if empty replace with ?
        predictions[model_name][i] = predictions[model_name][i].replace('-', ' ').replace('_.', ' ').replace('_', ' ').replace('\n', ' ').replace('!', ' ').strip()
        if predictions[model_name][i] == '':
            predictions[model_name][i] = "?"
        print("Post process prediction for", model_name,"is - ", predictions[model_name][i])
    # print("prediction for", model_name,"is - ", predictions[model_name])

for model_name in model_names:
    print("predictions for", model_name,
          "are",predictions[model_name],
          "bleu score for", model_name,"is - ", 
          bleu.compute(predictions=predictions[model_name], references=references[model_name]))



Found cached dataset squad (C:/Users/afons/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
In sample number 0 out of 2 samples in total


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In sample number 1 out of 2 samples in total


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Currently in sample  0
Pre process prediction for gpt2 is -  ____________________________________________________________________________________ I hope you enjoyed this summary of the
Post process prediction for gpt2 is -  I hope you enjoyed this summary of the
Pre process prediction for declare-lab/flan-alpaca-base is -  Denver Broncos
Post process prediction for declare-lab/flan-alpaca-base is -  Denver Broncos
Pre process prediction for facebook/opt-350m is -  __________.

The NFL has been the most
Post process prediction for facebook/opt-350m is -  The NFL has been the most
Currently in sample  1
Pre process prediction for gpt2 is -  ɪnɪn, as in "The
Post process prediction for gpt2 is -  ɪnɪn, as in "The
Pre process prediction for declare-lab/flan-alpaca-base is -  The NFL team represented the NFC at Super Bowl 50 was the Denver Broncos.
Post process prediction for declare-lab/flan-alpaca-base is -  The NFL team represented the NFC at Super Bowl 50 was the Denver Broncos.
Pre pro

# Using SpeechT5 for converting text-to-speech

Motivated by the success of T5 (Text-To-Text Transfer Transformer) in different natural language processing tasks, the unified-modal SpeechT5 framework explores encoder-decoder pre-training for self-supervised speech/text representation learning.

The model is again conveniently available through the HuggingFace Transformers library. The following example illustrates the use of the SpeechT5 model for generating a spectrogram from a textual input, together with a neural vocoder model for producing a speech signal.

More detailed information about SpeechT5 is available on a [tutorial on the HuggingFace blog](https://huggingface.co/blog/speecht5).

In [60]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
from IPython.display import Audio
import soundfile as sf
import torch

set_seed(42) # make results deterministic

model_T5 = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
processor_T5 = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

inputs = processor_T5(text="I have brought peace, freedom, justice and security to my new empire! Your new empire? Don't make me hurt you! Anakin, my allegiance is to the republic to DEMOCRACY! If you are not with me, then you are my enemy. Only a sith deals in absolutes... I will do what i must! You will try...", return_tensors="pt")
speaker_embeddings = torch.zeros((1, 512))
speech = model_T5.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
Audio("tts_example.wav", autoplay=True)

## Intermediate tasks:

* Connect the results from your answer to the previous intermediate task (i.e., conditioned language generation) to the SpeechT5 text-to-speech model, so as to produce speech outputs from the text generated by the model.
* Produce speech-based answers for the first 5 questions in the validation split from the SQuaD dataset.
* Connect also the results from your answer to the first intermediate task (i.e., automated speech recognition) to the SpeechT5 model and the LLM, so as to take spoken questions as input and produce a speech output.
* Collect small audio samples, with your own voice, for the first 5 questions in the validation split from the SQuaD dataset, and produce speech-based answers for these five questions.


In [61]:
from num2words import num2words

def convert_number_to_string(number):
    # Define a mapping for specific number conversions
    number_mapping = {
        "0": "zero",
        "10": "ten",
        "20": "twenty",
        "30": "thirty",
        "40": "forty",
        "50": "fifty",
        "60": "sixty",
        "70": "seventy",
        "80": "eighty",
        "80s" : "eighties",
        "90": "ninety",
        "100": "one hundred",
        "1000": "one thousand",
        "2000": "two thousand",
        "20th": "twentieth",
        "2019" : "twenty nineteen",
        # Add more mappings as needed
    }

    # Check if the number has a specific mapping
    if number in number_mapping:
        return number_mapping[number]
    else:
        return num2words(int(number))


In [62]:

beg = "I have  a question for you. The question is: "
middle = "To help you answer the question, I will give you some context. The context is: "
end = "Answer the question with 3 words or less. The answer is "
sample_size = 2
predictions_2 = []
# GENERATE TTS OUT OF SQUAD DATASET
# Only Alpaca Model as it is the best
for i in range(sample_size):
    references[model_name].append(answers_vec[i])
    # Create prompt for each model
    prompt = beg + questions_vec[i] + middle + contexts_vec[i] + end
    # Generate answer
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    generated_answer = generator(prompt, max_length=220, num_return_sequences=1)
    answer = generated_answer[0]['generated_text']
    predictions_2.append(answer)

# more post-processing
for i in range(sample_size):
    print("Currently in sample ", i)
    print("Pre process prediction for alpaca is - ", predictions_2[i])
    # if predictions[model_name][i] has -, _, or \n remove it then start from the first letter or number
    # if empty replace with ?
    # talked numbers!!!!
    predictions_2[i] = predictions_2[i].replace('-', ' ').replace('_.', ' ').replace('_', ' ').replace('\n', ' ').replace('!', ' ').strip()
    # i want to replace every number with its word
    predictions_2[i] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in predictions_2[i].split()])
    
    if predictions_2[i] == '':
        predictions_2[i] = "?"
    print("Post process prediction for alpaca is - ", predictions_2[i])
# print("prediction for", model_name,"is - ", predictions[model_name])
print("Predictions Post Processed")

for i in range(sample_size):
    print("Currently in sample ", i)
    print("Answer is - ", predictions_2[i])
    inputs = processor_T5(text=predictions_2[i], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

print("PITSTOP1")



Currently in sample  0
Pre process prediction for alpaca is -  Denver Broncos
Post process prediction for alpaca is -  Denver Broncos
Currently in sample  1
Pre process prediction for alpaca is -  The NFL team represented the NFC at Super Bowl 50 was the Denver Broncos.
Post process prediction for alpaca is -  The NFL team represented the NFC at Super Bowl fifty was the Denver Broncos.
Predictions Post Processed
Currently in sample  0
Answer is -  Denver Broncos


Currently in sample  1
Answer is -  The NFL team represented the NFC at Super Bowl fifty was the Denver Broncos.


PITSTOP1


In [63]:
#GENERATE TTS OUT OF CUSTOM AUDIOS

audio1, sr1 = librosa.load("helloHowRu.wav")
# resample using librosa
audio1 = librosa.resample(audio1, orig_sr = 22050, target_sr=16000)
audio2, sr2 = librosa.load("whereRuFrom.wav")
audio2 = librosa.resample(audio2, orig_sr = 22050, target_sr=16000)
audio_names = [audio1, audio2]
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Question is - ", transcription)
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(transcription, max_length=220, num_return_sequences=1)[0]
    print("Result is - ", result["generated_text"])
    answer_input = processor_T5(text=result["generated_text"], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   Hello, how are you?
Result is -  Hello, how are you?


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   Where are you from?
Result is -  I am from the United States.


In [64]:
# TTS from the first 5 (unique) questions from SQuaD 
# 1st Question - Which NFL team represented the AFC at Super Bowl 50?
# 2nd Question - Which NFL team represented the NFC at Super Bowl 50?
# 3rd Question - Where did Super Bowl 50 take place?
# 4th Question - Which NFL team won Super Bowl 50?
# 5th Question - What color was used to emphasize the 50th anniversary of the Super Bowl?
# 6th Question - What was the theme of Super Bowl 50?
# The group is using questions 1,3-6 and not question 2 due to audio processing challenges
audio_q1, _ = librosa.load("q1(1).wav")
audio_q1 = librosa.resample(audio_q1, orig_sr = 22050, target_sr=16000)
audio_q2, _ = librosa.load("q2.wav")
audio_q2 = librosa.resample(audio_q2, orig_sr = 22050, target_sr=16000)
audio_q3, _ = librosa.load("q3(1).wav")
audio_q3 = librosa.resample(audio_q3, orig_sr = 22050, target_sr=16000)
audio_q4, _ = librosa.load("q4.wav")
audio_q4 = librosa.resample(audio_q4, orig_sr = 22050, target_sr=16000)
audio_q5, _ = librosa.load("q5.wav")
audio_q5 = librosa.resample(audio_q5, orig_sr = 22050, target_sr=16000)
audio_names = [audio_q1, audio_q2, audio_q3, audio_q4, audio_q5]
# assume question mark
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # if transcription doesnt have a question mark, add one and remove the last character
    if transcription[-1] != "?":
        transcription = transcription[:-1]
        transcription = transcription + "?"
    print("Question is - ", transcription)
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(transcription, max_length=220, num_return_sequences=1)[0]
    # i want to replace every number with its word
    result["generated_text"] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in result["generated_text"].split()])
    print("Result is - ", result["generated_text"])
    answer_input = processor_T5(text=result["generated_text"], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   which NFL team represented the AFC at Super Bowl 15?
Result is -  The AFC represented the AFC at Super Bowl fifteen in Dallas, Texas.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   Where did Super Bowl 50 take place?
Result is -  Super Bowl fifty took place in Atlanta, Georgia.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   which NFL team won Super Bowl 50?
Result is -  The New England Patriots won Super Bowl 50.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What color was used to emphasize the 50th anniversary of the Super Bowl?
Result is -  The color used to emphasize the 50th anniversary of the Super Bowl was red.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What was the theme of Super Bowl 50?
Result is -  The theme of Super Bowl fifty was "The Power of Play".


# Main problem

Students are tasked with joining together the speech recognition, language understanding and generation, and text-to-speech models, in order to build a conversational spoken question answering approach.

* The method should take as input speech utterances with questions.
* The language understanding and generation component should use as input a transcription for the current speech utterance, and optionally also transcriptions from previous speech utterances (i.e., the conversation context).
* The language understanding and generation component can explore different strategies for improving answer quality:
  * Use of large LLMs trained with reinforcement learning from human feedback.
  * Prompting the language model with retrieved in-context examples.
  * Using parameter-efficient fine-ting with existing conversational question answering datasets (e.g., [the CoQA dataset](https://stanfordnlp.github.io/coqa/), available from HuggingFace datasets).
  * ...
* The text-to-speech component takes as input the results from language generation, and produces a speech output.
* Both the automated speech recognition and the text-to-speech components can explore different approaches, although students should attempt to justify their choices (e.g., if changing the automated speech recognition component, show that it achieves a lower WER).
* Collect small audio samples, with your own voice, for the first two instances in the CoQA validation split, and show the results produced by your method for these examples.

In [65]:
## FIRST 2 INSTANCES OF THE COQA DATASET
# 1st Question - What is the name of the movie?
# 2nd Question - Who wanted to build a bridge?
#load CoQA dataset
ds = load_dataset("coqa", split="train")
ds = ds.filter(lambda x: len(x["answers"]["answer_start"]) == 1)
ds = ds.filter(lambda x: x["answers"]["answer_start"][0] < 100)
ds = ds.filter(lambda x: x["answers"]["answer_start"][0] > 0)
ds = ds.filter(lambda x: len(x["questions"]) == 1)
print(ds[0].keys())  # Print the top-level keys of the first example
print(ds[0]["answers"].keys())  # Print the keys of the "answers" dictionary
print(ds[0]["questions"])  # Print the keys of the "questions" dictionary"])
print(ds[0]["answers"]["input_text"])  # Print the keys of the "questions" dictionary"])
print(ds[0]["answers"]["answer_start"])  # Print the keys of the "questions" dictionary"])
print(ds[0]["answers"]["answer_end"])  # Print the keys of the "questions" dictionary"])
starting_index = ds[0]["answers"]["answer_start"]
ending_index = ds[0]["answers"]["answer_end"]
string = ds[0]["story"]
string = string[starting_index[0]-1:ending_index[0]]
print(ds[0]["story"])
print(string)

Found cached dataset coqa (C:/Users/afons/.cache/huggingface/datasets/coqa/default/1.0.0/1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f)
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-3539b31ecb0fc300.arrow
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-b0d06394c70f4933.arrow
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-c60e053fc507f8d7.arrow
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-753637689047f3bb.arrow
dict_keys(['source', 'story', 'questions', 'answers'])
dict_keys(['input_text

In [66]:
# Questions as Speech Utterances
audio_CoQA_q1, _ = librosa.load("CoQA_1.wav")
audio_CoQA_q1 = librosa.resample(audio_CoQA_q1, orig_sr = 22050, target_sr=16000)
audio_CoQA_q2, _ = librosa.load("CoQA_2.wav")
audio_CoQA_q2 = librosa.resample(audio_CoQA_q2, orig_sr = 22050, target_sr=16000)
audio_names = [audio_CoQA_q1, audio_CoQA_q2]
inputs = processor_whisper(audio_CoQA_q1, return_tensors="pt")
input_features = inputs.input_features
generated_ids = model_whisper.generate(inputs=input_features)
transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Question is - ", transcription)
inputs = processor_whisper(audio_CoQA_q2, return_tensors="pt")
input_features = inputs.input_features
generated_ids = model_whisper.generate(inputs=input_features)
transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Question is - ", transcription)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is the name of the movie?
Question is -   Who wanted to build a bridge?


In [67]:
# Add your solutions to the exercises

beg = "Hey. I have a question for you my friend - "
middle = "This is the context so far - "
end = "What is the answer to my question in 5 words or less? The answer is - "
counter = 0
question_log = []
answer_log = []
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # if transcription doesnt have a question mark, add one and remove the last character
    if transcription[-1] != "?":
        transcription = transcription[:-1]
        transcription = transcription + "?"
    question_log.append(transcription)
    print("Question is - ", transcription)
    if counter > 0:
        prompt = beg + transcription + middle + answer_log[counter-1] + end
    else:
        prompt = beg + transcription + end
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(prompt, max_length=220, num_return_sequences=1)[0]
    # i want to replace every number with its word
    result["generated_text"] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in result["generated_text"].split()])
    answer_log.append(result["generated_text"])
    print("Result is - ", result["generated_text"])
    answer_input = processor_T5(text=result["generated_text"], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))
    counter += 1



It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is the name of the movie?
Result is -  The movie is "The Matrix".


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   Who wanted to build a bridge?
Result is -  Building Bridge.


In [68]:
for i in range(len(answer_log)):
    print("Question - ", question_log[i])
    print("Answer - ", answer_log[i])
    print("")

Question -   What is the name of the movie?
Answer -  The movie is "The Matrix".

Question -   Who wanted to build a bridge?
Answer -  Building Bridge.



In [69]:
## Personalized Conversation
# The goal is to prompt the alapaca model to make a correlation between its favourite band and the amount of listeners on a spanish speaking country
# Questions as Speech Utterances
audio_pr1, _ = librosa.load("pr1.wav")
audio_pr1 = librosa.resample(audio_pr1, orig_sr = 22050, target_sr=16000)
audio_pr2, _ = librosa.load("pr2.wav")
audio_pr2 = librosa.resample(audio_pr2, orig_sr = 22050, target_sr=16000)
audio_pr3, _ = librosa.load("pr3.wav")
audio_pr3 = librosa.resample(audio_pr3, orig_sr = 22050, target_sr=16000)
audio_pr4, _ = librosa.load("pr4.wav")
audio_pr4 = librosa.resample(audio_pr4, orig_sr = 22050, target_sr=16000)
audio_pr5, _ = librosa.load("pr5.wav")
audio_pr5 = librosa.resample(audio_pr5, orig_sr = 22050, target_sr=16000)
audio_pr6, _ = librosa.load("pr6.wav")
audio_pr6 = librosa.resample(audio_pr6, orig_sr = 22050, target_sr=16000)
audio_pr7, _ = librosa.load("pr7(1).wav")
audio_pr7 = librosa.resample(audio_pr7, orig_sr = 22050, target_sr=16000)
audio_names = [audio_pr1, audio_pr2, audio_pr3, audio_pr4, audio_pr5, audio_pr6, audio_pr7]
for audio in audio_names:
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Question is - ", transcription)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite 80s band?


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite music from that band?


...

Question is -   Do you think they had an important role in shaping that genre?


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is the largest Spanish-speaking country?


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is its capital?
Question is -   When was the last recorded concert of your favorite band in that country?


In [70]:
beg = "Hey. I have a question for you my friend - "
middle = "The question before hand was - "
middle2 = "And the previous answer was - "
end = "What is the answer to my question in 5 words or less? The answer is - "
counter = 0
question_log = []
answer_log = []
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Question is - ", transcription)

    # PRE-PROCESSING

    if transcription[-1] != "?":
        transcription = transcription[:-1]
        transcription = transcription + "?"
    question_log.append(transcription)
    
    ## ADDING CONTEXT
    ## CONTEXT = ANSWER TO PREVIOUS QUESTION
    if counter > 0:
        ## CONTEXT DEFINITON
        context = middle + question_log[counter-1] + middle2 + answer_log[counter-1]
        #---------------------------------------------------------------------------#
        prompt = beg + transcription + context + end
    else:
        prompt = beg + transcription + end
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(prompt, max_length=220, num_return_sequences=1)[0]

    # POST PROCESSING

    result["generated_text"] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in result["generated_text"].split()])
    answer_log.append(result["generated_text"])
    print("Result is - ", result["generated_text"])

    # TTS QUESTION

    question_input = processor_T5(text=transcription, return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(question_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

    # TTS ANSWER

    answer_input = processor_T5(text=result["generated_text"], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

    counter += 1

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite 80s band?
Result is -  My favorite 80s band is The Beatles.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite music from that band?
Result is -  My favorite music from The Beatles is "The Beatles" by The Beatles.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


...

Question is -   What is the largest Spanish-speaking country?
Result is -  The largest Spanish-speaking country is Spain.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is its capital?
Result is -  The capital of Spain is Madrid.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   When was the last recorded concert of your favorite band in that country?
Result is -  The last recorded concert of my favorite band in Spain was in Madrid, Spain on May 20th, 2019.


In [71]:
#-------------------------------------------------------------
# NO CONTEXT ADDED
#-------------------------------------------------------------

beg = "Hey. I have a question for you my friend - "
middle = "The question before hand was - "
middle2 = "And the previous answer was - "
end = "What is the answer to my question in 5 words or less? The answer is - "
counter = 0
question_log = []
answer_log = []
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Question is - ", transcription)

    # PRE-PROCESSING

    if transcription[-1] != "?":
        transcription = transcription[:-1]
        transcription = transcription + "?"
    question_log.append(transcription)
    
    prompt = beg + transcription + end
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(prompt, max_length=220, num_return_sequences=1)[0]

    # POST PROCESSING

    result["generated_text"] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in result["generated_text"].split()])
    answer_log.append(result["generated_text"])
    print("Result is - ", result["generated_text"])

    # TTS QUESTION

    question_input = processor_T5(text=transcription, return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(question_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

    # TTS ANSWER

    answer_input = processor_T5(text=result["generated_text"], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

    counter += 1

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite 80s band?
Result is -  My favorite 80s band is The Beatles.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite music from that band?
Result is -  My favorite music from that band is "The Beatles".


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


...

Question is -   What is the largest Spanish-speaking country?
Result is -  The largest Spanish-speaking country is Spain.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is its capital?
Result is -  The capital of the country is Washington D.C.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   When was the last recorded concert of your favorite band in that country?
Result is -  The last recorded concert of my favorite band in that country was in May, 2019.


In [72]:
#-------------------------------------------------------------
# NO PROMPT
#-------------------------------------------------------------
counter = 0
question_log = []
answer_log = []
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Question is - ", transcription)

    # PRE-PROCESSING

    if transcription[-1] != "?":
        transcription = transcription[:-1]
        transcription = transcription + "?"
    question_log.append(transcription)
    prompt = transcription
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(prompt, max_length=220, num_return_sequences=1)[0]

    # POST PROCESSING

    result["generated_text"] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in result["generated_text"].split()])
    answer_log.append(result["generated_text"])
    print("Result is - ", result["generated_text"])

    # TTS QUESTION

    question_input = processor_T5(text=transcription, return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(question_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

    # TTS ANSWER

    answer_input = processor_T5(text=result["generated_text"], return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

    counter += 1

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite 80s band?
Result is -  My favorite 80s band is The Beatles.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is your favorite music from that band?
Result is -  My favorite music from that band is "The Beatles".


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


...

Question is -   What is the largest Spanish-speaking country?
Result is -  The largest Spanish-speaking country is Mexico, with a population of over 1.4 billion people.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is its capital?
Result is -  The capital of the country is Washington D.C.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   When was the last recorded concert of your favorite band in that country?
Result is -  The last recorded concert of my favorite band in that country was in May of 2019.


In [73]:
## Reinforcement Learning with Human Feedback Sub-Task

In [74]:
from datasets import load_dataset
ds = load_dataset("coqa", split="train")
ds = ds.filter(lambda x: len(x["answers"]["answer_start"]) == 1)
ds = ds.filter(lambda x: x["answers"]["answer_start"][0] < 100)
ds = ds.filter(lambda x: x["answers"]["answer_start"][0] > 0)
ds = ds.filter(lambda x: len(x["questions"]) == 1)
print(ds[:5]["questions"])
question_temp = [q[0] for q in ds[:5]["questions"]]
print(question_temp)
answer_temp = [a["input_text"][0] for a in ds[:5]["answers"]]
print(answer_temp)
# Print COQA FIRST 5 QUESTIONS
for i in range(5):
    # print(ds[i]["story"])
    print(ds[i]["questions"][0])
    print(ds[i]["answers"]["input_text"][0])
    print()

Found cached dataset coqa (C:/Users/afons/.cache/huggingface/datasets/coqa/default/1.0.0/1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f)
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-3539b31ecb0fc300.arrow
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-b0d06394c70f4933.arrow
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-c60e053fc507f8d7.arrow
Loading cached processed dataset at C:\Users\afons\.cache\huggingface\datasets\coqa\default\1.0.0\1b03a32914e882ed315577005c472665e542419f910bab445815ad1929a7958f\cache-753637689047f3bb.arrow
[['What is the name of the movie?'], ['Who wanted to build a bridge?'], ['Why

In [75]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import AdamW
from torch.nn.utils.rnn import pad_sequence
import os

# Define your question-answering dataset
class QADataset(Dataset):
    def __init__(self, questions, answers, tokenizer):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.tokenizer.add_tokens(["<Q>", "<A>"])  # Add special tokens for questions and answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, index):
        question = self.questions[index]
        answer = self.answers[index]
        encoded_input = self.tokenizer.encode("<Q> " + question + " <A> " + answer, add_special_tokens=True)
        return torch.tensor(encoded_input)

# Set your questions and corresponding answers
# Extract to questions_ds the first 100 questions from the COQA dataset
# -----------------------------------------------------------------------
# CHANGE HERE TO TRAIN MODEL WITH MORE QUESTIONS
# -----------------------------------------------------------------------
sample_size = 2
questions_ds = [q[0] for q in ds[:sample_size]["questions"]]
answers_ds = [a["input_text"][0] for a in ds[:sample_size]["answers"]]


# Load the tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_tokens(["<Q>", "<A>"])
model = GPT2LMHeadModel.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# Create the question-answering dataset
dataset = QADataset(questions_ds, answers_ds, tokenizer)

# Set the padding token ID
tokenizer.pad_token = tokenizer.eos_token
padding_value = tokenizer.pad_token_id

# Assuming `dataset` is a list of tensors with varying lengths
input_sequences = [sample_item for sample_item in dataset]
padded_dataset = pad_sequence(input_sequences, batch_first=True, padding_value=padding_value)

# Define the RLHF training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

num_train_epochs = 10
per_device_train_batch_size = 8

for epoch in range(num_train_epochs):
    data_loader = DataLoader(padded_dataset, batch_size=per_device_train_batch_size, shuffle=True)
    for batch in data_loader:
        model.train()
        optimizer.zero_grad()
        input_ids = batch.to(device)
        
        # Generate model predictions
        outputs = model(input_ids=input_ids, labels=input_ids)
        logits = outputs.logits

        # Get the model's top-k predictions for each question
        k = 5  # Number of answers to consider
        _, topk_indices = torch.topk(logits[:, -1, :], k, dim=1)
        
        # Collect human-selected answers for each question
        selected_answers = []
        for question_ids, indices in zip(input_ids, topk_indices):
            question = tokenizer.decode(question_ids[1:-2], skip_special_tokens=True)
            print(f"Question: {question}")
            print("Top-K Answers:")
            for idx in indices:
                answer = tokenizer.decode([idx], skip_special_tokens=True)
                print(f"- {answer}")
            selected_answer = input("Select the answer (or enter a new answer): ")
            selected_answers.append(selected_answer)
            print()

        
        # Update the answers with RLHF
        updated_answers = selected_answers
        
        # Convert the updated answers back to tensors
        updated_input_ids = []
        for question, answer in zip(questions_ds, updated_answers):
            encoded_input = tokenizer.encode("<Q> " + question + " <A> " + answer, add_special_tokens=True)
            updated_input_ids.append(torch.tensor(encoded_input))
        updated_padded_dataset = pad_sequence(updated_input_ids, batch_first=True, padding_value=padding_value)
        updated_padded_dataset = updated_padded_dataset.to(device)
        
        # Update the model
        updated_outputs = model(input_ids=updated_padded_dataset[:, :-1], labels=updated_padded_dataset[:, 1:])
        updated_loss = updated_outputs.loss
        updated_loss.backward()
        optimizer.step()

# Save the fine-tuned model
output_dir = os.getcwd()
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



Question: What is the name of the movie? <A> Identity Thief is the
Top-K Answers:
- <Q>
- <A>
-  movie
-  in
-  film

Question: Who wanted to build a bridge? <A> John Roebling
Top-K Answers:
- <Q>
- <A>
- The
- A
- This

Question: What is the name of the movie? <A> Identity Thief is the
Top-K Answers:
- <Q>
- <A>
-  character
-  in
-  movie

Question: Who wanted to build a bridge? <A> John Roebling
Top-K Answers:
- <Q>
- <A>
- The
- A
- This

Question: Who wanted to build a bridge? <A> John Roebling
Top-K Answers:
- <Q>
- <A>
- The
- A
- This

Question: What is the name of the movie? <A> Identity Thief is the
Top-K Answers:
- <A>
- <Q>
-  character
-  movie
-  film

Question: What is the name of the movie? <A> Identity Thief is the
Top-K Answers:
- <Q>
- <A>
-  in
-  film
-  movie

Question: Who wanted to build a bridge? <A> John Roebling
Top-K Answers:
- <Q>
- <A>
- The
- A
- This

Question: What is the name of the movie? <A> Identity Thief is the
Top-K Answers:
- <Q>
- <A>
-  film
- 

('c:\\Users\\afons\\OneDrive\\Documentos\\4 Ano 2 Semestre\\PF\\Projeto\\PF\\Lab3\\tokenizer_config.json',
 'c:\\Users\\afons\\OneDrive\\Documentos\\4 Ano 2 Semestre\\PF\\Projeto\\PF\\Lab3\\special_tokens_map.json',
 'c:\\Users\\afons\\OneDrive\\Documentos\\4 Ano 2 Semestre\\PF\\Projeto\\PF\\Lab3\\vocab.json',
 'c:\\Users\\afons\\OneDrive\\Documentos\\4 Ano 2 Semestre\\PF\\Projeto\\PF\\Lab3\\merges.txt',
 'c:\\Users\\afons\\OneDrive\\Documentos\\4 Ano 2 Semestre\\PF\\Projeto\\PF\\Lab3\\added_tokens.json')

In [76]:
# Loading custom questions
audio_custom_1, _ = librosa.load("custom_1.wav")
audio_custom_1 = librosa.resample(audio_custom_1, orig_sr = 22050, target_sr=16000)
audio_custom_2, _ = librosa.load("custom_2.wav")
audio_custom_2 = librosa.resample(audio_custom_2, orig_sr = 22050, target_sr=16000)
audio_custom_3, _ = librosa.load("custom_3.wav")
audio_custom_3 = librosa.resample(audio_custom_3, orig_sr = 22050, target_sr=16000)
audio_names = [audio_custom_1, audio_custom_2, audio_custom_3]
question_log = []
answer_log = []
for audio in audio_names:
    #print(f"Current model is {model_name}...")
    inputs = processor_whisper(audio, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = model_whisper.generate(inputs=input_features)
    transcription = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print("Question is - ", transcription)

    # PRE-PROCESSING

    if transcription[-1] != "?":
        transcription = transcription[:-1]
        transcription = transcription + "?"
    question_log.append(transcription)
    generator = pipeline(model='declare-lab/flan-alpaca-base')
    result = generator(transcription, max_length=220, num_return_sequences=1)[0]

    # POST PROCESSING

    # result["generated_text"] = ' '.join([convert_number_to_string(number) if number.isdigit() else number for number in result["generated_text"].split()])
    answer_log.append(result["generated_text"])
    print("Result is - ", result["generated_text"])

    # TTS QUESTION

    question_input = processor_T5(text=transcription, return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(question_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   Who is Genki's Khan?
Result is -  Genki's Khan is a fictional character from the manga series "The Chronicles of Narnia". He is a renowned narrator and a leading figure in the Marvel Cinematic Universe.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   Which people might the first transatlantic flight?
Result is -  The first transatlantic flight was made by the United States in 1887.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Question is -   What is the first planet in the solar system?
Result is -  The first planet in the solar system is Mercury.


In [77]:
# Load the fine-tuned model
model_name = "gpt2"
output_dir = os.getcwd()
model = GPT2LMHeadModel.from_pretrained(output_dir)

# Set the model to evaluation mode
model.eval()

# Define a list of new questions for evaluation
# extract to new_question the questions created from questions_log
new_questions = question_log

# Generate answers for the new questions
generated_answers = []
for question in new_questions:
    input_ids = tokenizer.encode("<Q> " + question, add_special_tokens=True, return_tensors="pt").to(device)
    output_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)
    generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_answers.append(generated_answer)

# Print the generated answers
for question, generated_answer in zip(new_questions, generated_answers):
    print(f"Question: {question}")
    print(f"Generated Answer: {generated_answer}")
    print()

# Speak the answers
for answer in generated_answers:
    answer_input = processor_T5(text=answer, return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    speech = model_T5.generate_speech(answer_input["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write("tts_example.wav", speech.numpy(), samplerate=16000)
    display(Audio("tts_example.wav", autoplay=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question:  Who is Genki's Khan?
Generated Answer: <Q> Who is Genki's Khan? <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <A> <A> <A> <A> <A> <A>

Question:  Which people might the first transatlantic flight?
Generated Answer: <Q> Which people might the first transatlantic flight? <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <A> <A> <A> <A> <A> <A> <A> <A> <A> <A> <A>

Question:  What is the first planet in the solar system?
Generated Answer: <Q> What is the first planet in the solar system? <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <Q> <A> <A> <A> <A> <A> <A>

